In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import getpass
from pyspark.sql.types import StructField, StructType, \
    IntegerType, StringType, FloatType

In [2]:
username = getpass.getuser()
print(username)

band


In [3]:
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | section17'). \
    master('yarn'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-12-29 13:37:33,827 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [5]:
spark.sql("SHOW DATABASES")

2021-12-29 13:38:16,223 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2021-12-29 13:38:16,225 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


namespace
band_retail
company
default
hr_db
ordertest
retail_db


In [6]:
spark.sql("USE band_retail")

2021-12-29 13:38:25,158 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


""


In [7]:
spark.sql("SHOW TABLES")

namespace,tableName,isTemporary
band_retail,customers,false
band_retail,employees,false
band_retail,order_items,false
band_retail,orders,false


In [8]:
spark.sql("LOAD DATA LOCAL INPATH '/data/retail_db/orders/part-00000' OVERWRITE INTO TABLE orders")

2021-12-29 13:38:38,747 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2021-12-29 13:38:38,815 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2021-12-29 13:38:38,816 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2021-12-29 13:38:38,817 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


""


In [9]:
orders = spark.sql("SELECT * FROM orders")

In [10]:
order_items = spark.sql("SELECT * FROM order_items")

In [11]:
orders.select('*').show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [12]:
order_items.select('*').show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [13]:
orders_join = orders.join(
    order_items, 
    on=orders.order_id == order_items.order_item_order_id
)

In [14]:
orders_join2 = orders.join(
    order_items, 
    on=orders['order_id'] == order_items['order_item_order_id'],
    how='inner'
)

In [15]:
orders_join.select("*").show()

+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_id|          order_date|order_customer_id|   order_status|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+--------+--------------------+-----------------+---------------+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|            1|                  1|                  957|                  1|             299.98|                  299.98|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|

In [16]:
orders.count()

68883

In [17]:
order_items.count()

172198

In [18]:
orders_join.count()

172198

In [19]:
orders. \
    join(
        order_items, 
        on=orders['order_id'] == order_items['order_item_order_id'],
        how='inner'
    ). \
    select(orders['*'], order_items['order_item_subtotal']).show()

+--------+--------------------+-----------------+---------------+-------------------+
|order_id|          order_date|order_customer_id|   order_status|order_item_subtotal|
+--------+--------------------+-----------------+---------------+-------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|             299.98|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|             129.99|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|              250.0|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|             199.99|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|             199.92|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|              150.0|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|             299.95|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|              49.98|
|       5|2013-07-25 00:00:...|            11318|     

In [20]:
orders. \
    join(
        order_items, 
        on=orders['order_id'] == order_items['order_item_order_id']
    ). \
    select(orders['order_id'], orders['order_date'], orders['order_status'], order_items['order_item_subtotal']). \
    show()

+--------+--------------------+---------------+-------------------+
|order_id|          order_date|   order_status|order_item_subtotal|
+--------+--------------------+---------------+-------------------+
|       1|2013-07-25 00:00:...|         CLOSED|             299.98|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             129.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|              250.0|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             199.99|
|       4|2013-07-25 00:00:...|         CLOSED|             199.92|
|       4|2013-07-25 00:00:...|         CLOSED|              150.0|
|       4|2013-07-25 00:00:...|         CLOSED|             299.95|
|       4|2013-07-25 00:00:...|         CLOSED|              49.98|
|       5|2013-07-25 00:00:...|       COMPLETE|             129.99|
|       5|2013-07-25 00:00:...|       COMPLETE|             299.98|
|       5|2013-07-25 00:00:...|       COMPLETE|              99.96|
|       5|2013-07-25 00:00:...|       COMPLETE| 

In [21]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS customers(
        customer_city STRING,
        customer_email STRING,
        customer_fname STRING,
        customer_id INT,
        customer_lname STRING,
        customer_password STRING,
        customer_state STRING,
        customer_street STRING,
        customer_zipcode STRING
    )ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

2021-12-29 13:39:03,408 ERROR metastore.RetryingHMSHandler: AlreadyExistsException(message:Table customers already exists)
	at org.apache.hadoop.hive.metastore.HiveMetaStore$HMSHandler.create_table_core(HiveMetaStore.java:1416)
	at org.apache.hadoop.hive.metastore.HiveMetaStore$HMSHandler.create_table_with_environment_context(HiveMetaStore.java:1503)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at org.apache.hadoop.hive.metastore.RetryingHMSHandler.invokeInternal(RetryingHMSHandler.java:148)
	at org.apache.hadoop.hive.metastore.RetryingHMSHandler.invoke(RetryingHMSHandler.java:107)
	at com.sun.proxy.$Proxy33.create_table_with_environment_context(Unknown Source)
	at org.apache.hadoop.hive.metastore.HiveMetaStoreClient.create_table_with_environmen

""


In [22]:
spark.sql("LOAD DATA LOCAL INPATH '/data/retail_db/customers/part-00000' INTO TABLE customers")

""


In [23]:
customers = spark.sql("SELECT * FROM customers")

In [24]:
customers.printSchema()

root
 |-- customer_city: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [25]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [26]:
customers.show()

+-------------+--------------+--------------+-----------+--------------+--------------------+--------------+---------------+----------------+
|customer_city|customer_email|customer_fname|customer_id|customer_lname|   customer_password|customer_state|customer_street|customer_zipcode|
+-------------+--------------+--------------+-----------+--------------+--------------------+--------------+---------------+----------------+
|            1|       Richard|     Hernandez|       null|     XXXXXXXXX|  6303 Heather Plaza|   Brownsville|             TX|           78521|
|            2|          Mary|       Barrett|       null|     XXXXXXXXX|9526 Noble Embers...|     Littleton|             CO|           80126|
|            3|           Ann|         Smith|       null|     XXXXXXXXX|3422 Blue Pioneer...|        Caguas|             PR|           00725|
|            4|          Mary|         Jones|       null|     XXXXXXXXX|  8324 Little Common|    San Marcos|             CA|           92069|
|     

In [27]:
customer_order_details = customers.join(
    orders, 
    on=customers['customer_id'] == orders['order_customer_id'],
    how='inner'
)

In [28]:
customers.select('customer_fname').show()

+--------------+
|customer_fname|
+--------------+
|     Hernandez|
|       Barrett|
|         Smith|
|         Jones|
|        Hudson|
|         Smith|
|        Wilcox|
|         Smith|
|         Perez|
|         Smith|
|       Huffman|
|         Smith|
|       Baldwin|
|         Smith|
|          Luna|
|         Smith|
|      Robinson|
|         Smith|
|      Mitchell|
|         Ellis|
+--------------+
only showing top 20 rows



In [29]:
customers.groupBy('customer_fname').agg(count('*').alias('count_fname')).sort(asc('customer_fname')).show()

+--------------+-----------+
|customer_fname|count_fname|
+--------------+-----------+
|        Abbott|          2|
|       Acevedo|         10|
|        Acosta|         14|
|         Adams|         44|
|        Adkins|         12|
|       Aguilar|         16|
|       Aguirre|          6|
|     Alexander|         24|
|           Ali|          6|
|         Allen|         58|
|       Allison|          6|
|      Alvarado|         12|
|       Alvarez|         24|
|      Andersen|          4|
|      Anderson|         66|
|       Andrade|         14|
|       Andrews|         26|
|       Anthony|          4|
|        Archer|          8|
|      Arellano|          8|
+--------------+-----------+
only showing top 20 rows



In [30]:
path = f"/user/{username}/warehouse/15_first.csv"
print(path)

/user/band/warehouse/15_first.csv


In [31]:
spark.read.csv(path=path, header=True, inferSchema=True).show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [32]:
employeesSchema = StructType([
    StructField("employee_id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("salary", FloatType()),
    StructField("nationality", StringType())
])

In [33]:
spark.catalog.listTables()

[Table(name='customers', database='band_retail', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employees', database='band_retail', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_items', database='band_retail', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders', database='band_retail', description=None, tableType='MANAGED', isTemporary=False)]

In [ ]:
spark.catalog.createTable("employees", schema=employeesSchema)

In [ ]:
employees = [(1, "Scott", "Tiger", 1000.0, "united states"),
             (2, "Henry", "Ford", 1250.0, "India"),
             (3, "Nick", "Junior", 750.0, "united KINGDOM"),
             (4, "Bill", "Gomes", 1500.0, "AUSTRALIA")
            ]

In [ ]:
employeesDF = spark.createDataFrame(employees,
    schema="""employee_id INT, first_name STRING, last_name STRING,
              salary FLOAT, nationality STRING
           """
)

In [ ]:
employeesDF.write.insertInto('employees', overwrite=True)

In [ ]:
spark.read.table('employees').show()

In [ ]:
spark.sql('SHOW TABLES').show()

In [ ]:
orders.createTempView('ordersTempView')

In [ ]:
orders.printSchema()

In [ ]:
orders.select(date_format('order_date','yyyy-MM-dd').alias('date')).sort(desc('date')).show()

In [ ]:
orders.select('order_date').orderBy(asc('order_date')).show()

In [ ]:
orders.printSchema()

In [35]:
x = orders.groupBy('order_date').agg(count(col('order_date')).alias('cout_date'))

In [36]:
x.select('cout_date').sort(desc('cout_date')).show(1)

+---------+
|cout_date|
+---------+
|      347|
+---------+
only showing top 1 row



In [37]:
x.select(max('cout_date').alias('max_date'), min('cout_date').alias('min_date')).show()

+--------+--------+
|max_date|min_date|
+--------+--------+
|     347|      73|
+--------+--------+



In [ ]:
x.filter(col('cout_date') == 73).show()

In [ ]:
orders.withColumn('order_month', date_format('order_date', 'yyyyMM')).write.partitionBy('order_month').parquet(f'/user/{username}/retail_db/orders_part')

In [ ]:
spark.read.parquet(f'/user/{username}/retail_db/orders_part/order_month=201407').show()